### [Harris Corner Detection](https://docs.opencv.org/3.4/dc/d0d/tutorial_py_features_harris.html)

Okay, Corners are good features? But how do we find them?

#### Goal

In this chapter,

* We will understand the concepts behind Harris Corner Detection.
* We will see the functions: [cv.cornerHarris()](https://docs.opencv.org/3.4/dd/d1a/group__imgproc__feature.html#gac1fc3598018010880e370e2f709b4345), [cv.cornerSubPix()](https://docs.opencv.org/3.4/dd/d1a/group__imgproc__feature.html#ga354e0d7c86d0d9da75de9b9701a9a87e)

#### Theory
마지막 장에서 우리는 모서리가 모든 방향의 강도가 큰 이미지의 영역임을 확인 했습니다. 이 모서리를 찾기 위한 초기 시도 중 하나는 **Chris Harris & Mike Stephens**가 1988년 **Combined Corner and Edge Detector**라는 논문에서 해리스 코너 탐지기라고 불렀던 것입니다. 그는 이 간단한 생각을 수학적 형태로 취했습니다. 기본적으로 모든 방향에서 \\((u, v)\\) 변위에 대한 강도의 차이를 찾습니다. 이것은 다음과 같이 표현됩니다

\\(E(u,v) = \sum_{x,y} \underbrace{w(x,y)}_\text{window function} \, [\underbrace{I(x+u,y+v)}_\text{shifted intensity}-\underbrace{I(x,y)}_\text{intensity}]^2\\)

윈도우 함수는 직사각형 윈도우이거나 아래 픽셀에 가중치를 주는 가우시안 윈도우입니다.

코너 검출을 위해이 함수 \\(E(u, v)\\)를 최대화 해야 합니다. 즉, 우리는 두 번째 용어를 최대화 해야 합니다. 위의 방정식에 Taylor Expansion을 적용하고 몇 가지 수학적 단계를 사용하여 (전체 파생에 대해 원하는 표준 텍스트 책을 참조하십시오), 최종 방정식을 다음과 같이 얻습니다.  
\\(E(u,v) \approx \begin{bmatrix} u & v \end{bmatrix} M \begin{bmatrix} u \\ v \end{bmatrix}\\)  
where  
\\(M = \sum_{x,y} w(x,y) \begin{bmatrix}I_x I_x & I_x I_y \\ I_x I_y & I_y I_y \end{bmatrix}\\)

여기서, \\(I_x\\) and \\(I_y \\)는 각각 x와 y 방향의 이미지 도함수이다. ([cv.Sobel()](https://docs.opencv.org/3.4/d4/d86/group__imgproc__filter.html#gacea54f142e81b6758cb6f375ce782c8d)을 사용하면 쉽게 찾을 수있다).

그런 다음 주요 부분이 옵니다. 이 후, 그들은 점수를 만들었습니다. 기본적으로 방정식은 창에 모서리가 있는지 여부를 결정합니다.  
\\(R = det(M) - k(trace(M))^2\\)

where

* \\(det(M)=λ_1λ_2\\)
* \\(trace(M)=λ_1+λ_2\\)
* \\(λ_1\\) and \\(λ_2\\) are the eigen values of \\(M\\)

따라서 이러한 고유 값의 값은 영역이 모서리, 모서리 또는 평면인지 여부를 결정합니다.

* \\(λ_1\\)과  \\(λ_2\\)가 작을 때 발생하는 \\(|R|\\)가 작으면 이 영역은 평평합니다.
* \\(R <0\\) 일 때 \\(λ_1 >> λ_2\\) 일 때 또는 그 반대 일 때 영역은 가장자리입니다.
* \\(R\\)이 클 때, \\(λ_1\\)과 \\(λ_2\\)가 크고 \\(λ_1 ~ λ_2\\) 일 때 발생하는 영역은 구석입니다.
다음과 같이 멋진 그림으로 표현할 수 있습니다.
![](harris_region.jpg)

따라서 Harris Corner Detection의 결과는 이러한 점수를 가진 회색 음영 이미지입니다. 적합한 임계 값은 이미지의 모서리를 제공합니다. 우리는 간단한 이미지로 그것을 할 것입니다

#### OpenCV의 Harris Corner Detector
OpenCV는 이 목적으로 cv.cornerHarris () 함수를 가지고 있습니다. 그것의 논쟁은 :

* img - 입력 이미지. 그레이 스케일 및 float32 유형이어야합니다.
* blockSize - 코너 감지를 위해 고려 된 이웃의 크기입니다.
* ksize - 사용 된 소벨 유도체의 애퍼 추어 매개 변수입니다.
* k - 방정식의 해리스 탐지기없는 매개 변수.

In [24]:
import numpy as np
import cv2 as cv
import bonghanUtil as util

filename = 'chessboard.png'
img      = cv.imread(filename)
img      = cv.resize(img, None, fx = 0.2, fy = 0.2, interpolation = cv.INTER_CUBIC)
gray     = cv.cvtColor(img,cv.COLOR_BGR2GRAY)
gray     = np.float32(gray)
dst      = cv.cornerHarris(gray,2,3,0.04)

#result is dilated for marking the corners, not important
dst2 = cv.dilate(dst, None)

# Threshold for an optimal value, it may vary depending on the image.
img[dst2 > 0.01 * dst2.max()]=[0,0,255]
util.saveImage('result_chessboard.jpg', img)
util.showImage(img, 'dst')

filename = 'checkboard_b.jpg'
img      = cv.imread(filename)
#img      = cv.resize(img, None, fx = 0.2, fy = 0.2, interpolation = cv.INTER_CUBIC)
gray     = cv.cvtColor(img,cv.COLOR_BGR2GRAY)
gray     = np.float32(gray)
dst      = cv.cornerHarris(gray,2,3,0.04)

#result is dilated for marking the corners, not important
dst2 = cv.dilate(dst, None)

# Threshold for an optimal value, it may vary depending on the image.
img[dst2 > 0.01 * dst2.max()]=[0,0,255]
util.saveImage('result_checkboard_b.jpg', img)
util.showImage(img, 'dst')

filename = 'blox.jpg'
img      = cv.imread(filename)
#img      = cv.resize(img, None, fx = 0.2, fy = 0.2, interpolation = cv.INTER_CUBIC)
gray     = cv.cvtColor(img,cv.COLOR_BGR2GRAY)
gray     = np.float32(gray)
dst      = cv.cornerHarris(gray,2,3,0.04)

#result is dilated for marking the corners, not important
dst2 = cv.dilate(dst, None)

# Threshold for an optimal value, it may vary depending on the image.
img[dst2 > 0.01 * dst2.max()]=[0,0,255]
util.saveImage('result_blox.jpg', img)
util.showImage(img, 'dst')

![](result_chessboard.jpg)
![](result_checkboard_b.jpg)
![](result_blox.jpg)

In [ ]:
# Test code
import numpy as np
import cv2 as cv
import bonghanUtil as util

filename = 'test_chessboard2.png'
img      = cv.imread(filename)
gray     = cv.cvtColor(img, cv.COLOR_BGR2GRAY)

# find Harris corners
gray     = np.float32(gray)
dst      = cv.cornerHarris(gray, 2, 3, 0.04)
dst      = cv.dilate(dst, None)
ret, dst = cv.threshold(dst, 0.01 * dst.max(), 255, 0)
dst      = np.uint8(dst)

# find centroids
ret, labels, stats, centroids = cv.connectedComponentsWithStats(dst)

# define the criteria to stop and refine the corners
criteria = (cv.TERM_CRITERIA_EPS + cv.TERM_CRITERIA_MAX_ITER, 100, 0.001)
corners = cv.cornerSubPix(gray, np.float32(centroids), (5, 5), (-1, -1), criteria)

# Now draw them
res = np.hstack((centroids,corners))
res = np.int0(res)
img[res[:,1],res[:,0]] = [0,0,255]
img[res[:,3],res[:,2]] = [0,255,0]

util.showImage(img, 'subpixel5.png')

#### 서브 픽셀 정확도가 있는 모서리
경우에 따라 최대 정확도로 모서리를 찾아야 할 수도 있습니다. OpenCV에는 [cv.cornerSubPix()](https://docs.opencv.org/3.4/dd/d1a/group__imgproc__feature.html#ga354e0d7c86d0d9da75de9b9701a9a87e) 함수가 있습니다.이 함수 는 서브 픽셀 정확도로 감지 된 모서리를 더욱 세련시킵니다. 아래는 그 예입니다. 평소와 같이 해리스 코너를 먼저 찾아야합니다. 그런 다음이 구석의 중심을지나칩니다 (모서리에 픽셀 무리가있을 수 있습니다). 해리스 모서리는 빨간색 픽셀로 표시되고 세련된 모서리는 녹색 픽셀로 표시됩니다. 이 함수의 경우 반복을 중지 할 기준을 정의해야합니다. 지정된 반복 횟수 또는 일정한 정확도가 달성 된 후 (둘 중 먼저 발생하는 시점)에 중지합니다. 우리는 모서리를 검색 할 이웃의 크기를 정의해야합니다.

In [6]:
import numpy as np
import cv2 as cv
import bonghanUtil as util

filename = 'chessboard2.png'
img      = cv.imread(filename)
gray     = cv.cvtColor(img, cv.COLOR_BGR2GRAY)

# find Harris corners
gray     = np.float32(gray)
dst      = cv.cornerHarris(gray, 2, 3, 0.04)
dst      = cv.dilate(dst, None)
ret, dst = cv.threshold(dst, 0.01 * dst.max(), 255, 0)
dst      = np.uint8(dst)

# find centroids
ret, labels, stats, centroids = cv.connectedComponentsWithStats(dst)

# define the criteria to stop and refine the corners
criteria = (cv.TERM_CRITERIA_EPS + cv.TERM_CRITERIA_MAX_ITER, 100, 0.001)
corners = cv.cornerSubPix(gray, np.float32(centroids), (5, 5), (-1, -1), criteria)

# Now draw them
res = np.hstack((centroids,corners))
res = np.int0(res)
img[res[:,1],res[:,0]] = [0,0,255]
img[res[:,3],res[:,2]] = [0,255,0]

cv.imwrite('subpixel5.png', img)
util.showImage(img, 'subpixel5.png')

Below is the result, where some important locations are shown in zoomed window to visualize:

![](subpixel5.png)